Copyright (C) 2023 Benjamin Futasz

In [ ]:
import pathlib

import fcecodec

py_scriptsdir = f"./PyScripts/"
fcec_scriptsdir = f"./scripts/"
fshtool = f"./fshtool"
unvivtool_script = f"./unvivtool_script.py"

fce_path_input = f"car.fce"
fce_path_output = f"car.fce"

fsh_path_input = f"file.fsh"

car00tga_input = f"car00.tga"
car00tga_output = f"car00.tga"

viv_source_dir = f"car_viv"

fce_path_colorsource = f"car.fce"

fce_version_target = 3

In [ ]:
if fce_version_target in ["4M", "4m"]:
    fce_version_target = 5
assert fce_version_target in [ 3, 4, 5]

def GetFceVersion(path):
    with open(path, "rb") as f:
        version = fcecodec.GetFceVersion(f.read(0x2038))
        assert version > 0
        return version

input_fce_version = GetFceVersion(fce_path_input)
print(input_fce_version)

In [ ]:
# Convert from FCE4M to FCE4, if applicable:
current_fce_version = GetFceVersion(fce_path_input)
if current_fce_version == 5 and fce_version_target < 5:
    pass
    !python "{fcec_scriptsdir}""bfut_PrintFceInfo.py" "{fce_path_input}"
    !echo "bfut_MergeParts (FceM to Fce4, keep version).py"
    !python "{fcec_scriptsdir}""bfut_MergeParts (FceM to Fce4, keep version).py" 1 0 1 "{fce_path_input}" "{fce_path_output}"
    !echo DONE "bfut_MergeParts (FceM to Fce4, keep version).py"
    !python "{fcec_scriptsdir}""bfut_SaveFceAsFce4.py" "{fce_path_output}" "{fce_path_output}"
    !python "{fcec_scriptsdir}""bfut_CopyCarColors.py" "{fce_path_colorsource}" "{fce_path_output}"

    # !python "{fcec_scriptsdir}""bfut_PrintFceInfo.py" "{fce_path_output}"
    fce_path_input = fce_path_output

In [ ]:
# Convert from FCE4 to FCE3, if applicable
current_fce_version = GetFceVersion(fce_path_input)
if current_fce_version == 4 and fce_version_target < 4:
    pass
    !python "{fcec_scriptsdir}""bfut_PrintFceInfo.py" "{fce_path_input}"
    !echo "bfut_MergeParts (Fce4 to Fce3, keep version).py"
    !python "{fcec_scriptsdir}""bfut_MergeParts (Fce4 to Fce3, keep version).py" 1 opaque "{fce_path_input}" "{fce_path_output}"
    !echo "DONE bfut_MergeParts (Fce4 to Fce3, keep version).py"
    !python "{fcec_scriptsdir}""bfut_SortPartsToFce3Order (keep fce version).py" "{fce_path_output}" "{fce_path_output}"
    !python "{fcec_scriptsdir}""bfut_ConvertDummiesToFce3.py" "{fce_path_output}" "{fce_path_output}"
    !python "{fcec_scriptsdir}""bfut_SaveFceAsFce3.py" "{fce_path_output}" "{fce_path_output}"
    # !python "{fcec_scriptsdir}""bfut_PrintFceInfo.py" "{fce_path_output}"

with open(fce_path_output, "rb") as f:
    assert fcecodec.ValidateFce(f.read()) == 1

Convert FSH to TGA

In [ ]:
if type(fsh_path_input) is str and len(fsh_path_input) > 0:
    pass
    p = pathlib.Path(fsh_path_input)
    car00tga_input = p.with_suffix(".tga")

    !{fshtool} "{fsh_path_input}"
    BMP = (p.parent / p.stem / "0000").with_suffix(".BMP")
    BMPA = (p.parent / p.stem / "0000-a").with_suffix(".BMP")
    !convert "{BMP}" "{BMPA}" -auto-orient -alpha off -compose CopyOpacity -composite "{car00tga_input}"

Convert TGA alpha channel to target version

In [ ]:
!cp "{car00tga_input}" "{car00tga_output}"
!python "{py_scriptsdir}""bfut_TextureRotator/bfut_TextureRotator (flip vertically).py" "{car00tga_output}" "{car00tga_output}"

if fce_version_target == 3:
    if input_fce_version == 4:
        pass
        !python "{py_scriptsdir}""bfut_NfsTgaConverter/bfut_NfsTgaConverter (HSto3).py" "{car00tga_output}" "{car00tga_output}"
    elif input_fce_version == 5:
        pass
        !python "{py_scriptsdir}""bfut_NfsTgaConverter/bfut_NfsTgaConverter (Mto3).py" "{car00tga_output}" "{car00tga_output}"

Create VIV archive

In [ ]:
!python {unvivtool_script} e "{viv_source_dir}"